In [2]:
# Практика 2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import log_loss

# 1. Загрузка данных и разделение
try:
    data = pd.read_csv('gbm-data.csv')
except FileNotFoundError:
    print("Ошибка: Файл 'gbm-data.csv' не найден в текущей директории.")
    exit(1)

X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.8, random_state=241
)

# 2. Обучение Gradient Boosting для разных learning_rate
learning_rates = [1, 0.5, 0.3, 0.2, 0.1]
results = {}

for lr in learning_rates:
    print(f"\nОбучение с learning_rate={lr}")
    gb = GradientBoostingClassifier(
        n_estimators=250,
        learning_rate=lr,
        random_state=241,
        verbose=True
    )
    gb.fit(X_train, y_train)
    
    # Получаем предсказания на каждой итерации
    train_staged = gb.staged_decision_function(X_train)
    test_staged = gb.staged_decision_function(X_test)
    
    train_losses = []
    test_losses = []
    
    # Вычисляем log-loss для каждой итерации
    for train_pred, test_pred in zip(train_staged, test_staged):
        # Применяем сигмоиду
        train_prob = 1 / (1 + np.exp(-train_pred))
        test_prob = 1 / (1 + np.exp(-test_pred))
        
        # Убеждаемся, что массивы имеют правильную форму
        if train_prob.ndim > 1:
            train_prob = train_prob[:, 0]  # Для бинарной классификации берем первый столбец
        if test_prob.ndim > 1:
            test_prob = test_prob[:, 0]
        
        # Вычисляем log-loss
        train_loss = log_loss(y_train, train_prob)
        test_loss = log_loss(y_test, test_prob)
        
        train_losses.append(train_loss)
        test_losses.append(test_loss)
    
    results[lr] = {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'min_test_loss': min(test_losses),
        'best_iter': np.argmin(test_losses) + 1  # +1 так как итерации начинаются с 1
    }

# 3. Анализ переобучения для learning_rate=0.2
test_losses_lr02 = results[0.2]['test_losses']
train_losses_lr02 = results[0.2]['train_losses']

# Проверяем, начинает ли тестовая ошибка расти при продолжающемся снижении обучающей ошибки
overfitting_detected = False
for i in range(1, len(test_losses_lr02)):
    if test_losses_lr02[i] > test_losses_lr02[i-1] and train_losses_lr02[i] < train_losses_lr02[i-1]:
        overfitting_detected = True
        break

print(f"\nХарактер графика качества на тестовой выборке: {'overfitting' if overfitting_detected else 'underfitting'}")

# 4. Результаты для learning_rate=0.2
best_iter_lr02 = results[0.2]['best_iter']
min_test_loss_lr02 = results[0.2]['min_test_loss']
print(f"\nДля learning_rate = 0.2:")
print(f"Минимальное значение log-loss на тестовой выборке: {min_test_loss_lr02:.2f}")
print(f"Номер итерации: {best_iter_lr02}")

# 5. RandomForest с количеством деревьев = best_iter_lr02
rf = RandomForestClassifier(
    n_estimators=best_iter_lr02,
    random_state=241
)
rf.fit(X_train, y_train)

# Получаем вероятности принадлежности к классу 1
rf_probs = rf.predict_proba(X_test)[:, 1]
rf_log_loss = log_loss(y_test, rf_probs)
print(f"\nLog-loss для RandomForest: {rf_log_loss:.2f}")


Обучение с learning_rate=1
      Iter       Train Loss   Remaining Time 
         1           1.0190           14.09s
         2           0.9192            9.87s
         3           0.8272            8.33s
         4           0.7834            7.57s
         5           0.7109            7.14s
         6           0.6368            6.82s
         7           0.5797            6.57s
         8           0.5610            6.38s
         9           0.5185            6.23s
        10           0.4984            6.12s
        20           0.1999            5.46s
        30           0.1313            5.09s
        40           0.0790            4.79s
        50           0.0511            4.54s
        60           0.0352            4.29s
        70           0.0245            4.04s
        80           0.0162            3.81s
        90           0.0114            3.58s
       100           0.0077            3.35s
       200           0.0002            1.11s

Обучение с learning_rate=